In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import sys,os
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
def mannwhitney(x,y):
    #x,y are data frames of same dimensionality
    junk=[]
    for row in range(x.shape[0]):
        series1=np.array(x.iloc[row])
        series2=np.array(y.iloc[row])
        z_stat, p_val = stats.mannwhitneyu(series1, series2)
        junk.append(p_val*2.0)
    #p-value is for a one-sided hypothesis, to get the two-sided p-value multiply the returned p-value by 2.
    result=pd.DataFrame(np.array(junk),index=x.index)
    return result

In [4]:
def wilcoxon(x):
    #x is data frame of changes (differences)
    junk=[]
    for row in range(x.shape[0]):
        series1=np.array(x.iloc[row])
        z_stat, p_val = stats.wilcoxon(series1)
        junk.append(p_val)
    #returns the two-sided p-value for the test.
    result=pd.DataFrame(np.array(junk),index=x.index)
    return result

In [5]:
gcms =  [ 'gfdl-esm2g','canesm2','cnrm-cm5','csiro-mk3-6-0','inmcm4','miroc5','mpi-esm-lr','mri-cgcm3','ccsm4',
         'noresm1-m','bcc-csm1-1','ipsl-cm5a-lr','ipsl-cm5a-mr','gfdl-cm3','gfdl-esm2m' ]
#gcms =  [ 'gfdl-esm2g' ]

In [6]:
scen = 'rcp45'
#futperiod='2070-2099'
futperiod='2040-2069'
v='pr'

# dir is location of files like gcm_scen_v_2040-2069_stats.csv. Output also goes here.

#columns in input files contain data and stats, labeled:
                #ID,Lon,Lat,nyears,onset_mx_mn,onset_dy_mn,end_mx_mn,end_dy_mn,min_mn_mn,duration_mn,intensity_mn,onset_mx_sd,onset_dy_sd,end_mx_sd,end_dy_sd,min_mn_sd,duration_sd,intensity_sd

In [7]:
#windows requires some different handling of paths with backslashes
dir=os.path.normpath("X:/projects/nicaragua/msd/timeseries_results")

In [8]:
#set up data frames for output - historical for intercomparison with obs, summary changes
hist_onset_mx_15=pd.DataFrame(index=range(303))
hist_onset_dy_15=pd.DataFrame(index=range(303))
hist_min_mn_15=pd.DataFrame(index=range(303))
hist_duration_15=pd.DataFrame(index=range(303))
hist_intensity_15=pd.DataFrame(index=range(303))
hist_end_mx_15=pd.DataFrame(index=range(303))
hist_end_dy_15=pd.DataFrame(index=range(303))
histmean_all=pd.DataFrame(index=range(303))

futchange_onset=pd.DataFrame(index=range(303))
futchange_end=pd.DataFrame(index=range(303))
futchange_duration=pd.DataFrame(index=range(303))
futchange_min=pd.DataFrame(index=range(303))
futchange_max1=pd.DataFrame(index=range(303))
futchange_max2=pd.DataFrame(index=range(303))
futchange_intensity=pd.DataFrame(index=range(303))
futchanges_all=pd.DataFrame(index=range(303))

In [9]:
for gcm in gcms:
    #read in historical and future files - one row per cell, one col per year
    inf_fut="%s\%s_%s_%s_%s_stats.csv" % (dir,gcm,scen,v,futperiod)
    print 'opening ' + inf_fut
    df_fut = pd.read_csv(inf_fut)
    inf_his="%s\%s_%s_%s_%s_stats.csv" % (dir,gcm,'historical',v,'1970-1999')
    print 'opening ' + inf_his
    df_his = pd.read_csv(inf_his)
    
    #get cell, lon, lat from first file
    if gcm == gcms[0]:
        cellinfo=df_his[['ID','Lon','Lat']]
    
    #concatenate statistics to new data frames. 
    dur=df_his['end_dy_mn']-df_his['onset_dy_mn']
    durfut=df_fut['end_dy_mn']-df_fut['onset_dy_mn']
    intens=((df_his['end_mx_mn']+df_his['onset_mx_mn'])/2.0)-df_his['min_mn_mn']
    intensfut=((df_fut['end_mx_mn']+df_fut['onset_mx_mn'])/2.0)-df_fut['min_mn_mn']
    hist_onset_mx_15=pd.concat([hist_onset_mx_15,df_his['onset_mx_mn']],axis=1,join_axes=[hist_onset_mx_15.index])
    hist_onset_dy_15=pd.concat([hist_onset_dy_15,df_his['onset_dy_mn']],axis=1,join_axes=[hist_onset_dy_15.index])
    hist_end_mx_15=pd.concat([hist_end_mx_15,df_his['end_mx_mn']],axis=1,join_axes=[hist_end_mx_15.index])
    hist_end_dy_15=pd.concat([hist_end_dy_15,df_his['end_dy_mn']],axis=1,join_axes=[hist_end_dy_15.index])
    hist_min_mn_15=pd.concat([hist_min_mn_15,df_his['min_mn_mn']],axis=1,join_axes=[hist_min_mn_15.index])
    hist_duration_15=pd.concat([hist_duration_15,dur],axis=1,join_axes=[hist_duration_15.index])
    hist_intensity_15=pd.concat([hist_intensity_15,intens],axis=1,join_axes=[hist_intensity_15.index])
    futchange_onset=pd.concat([futchange_onset,df_fut['onset_dy_mn']-df_his['onset_dy_mn']],axis=1,join_axes=[futchange_onset.index])
    futchange_end=pd.concat([futchange_end,df_fut['end_dy_mn']-df_his['end_dy_mn']],axis=1,join_axes=[futchange_end.index])
    futchange_duration=pd.concat([futchange_duration,durfut-dur],axis=1,join_axes=[futchange_duration.index])
    futchange_intensity=pd.concat([futchange_intensity,intensfut-intens],axis=1,join_axes=[futchange_intensity.index])
    futchange_min=pd.concat([futchange_min,df_fut['min_mn_mn']-df_his['min_mn_mn']],axis=1,join_axes=[futchange_min.index])

opening X:\projects\nicaragua\msd\timeseries_results\gfdl-esm2g_rcp45_pr_2040-2069_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\gfdl-esm2g_historical_pr_1970-1999_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\canesm2_rcp45_pr_2040-2069_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\canesm2_historical_pr_1970-1999_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\cnrm-cm5_rcp45_pr_2040-2069_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\cnrm-cm5_historical_pr_1970-1999_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\csiro-mk3-6-0_rcp45_pr_2040-2069_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\csiro-mk3-6-0_historical_pr_1970-1999_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\inmcm4_rcp45_pr_2040-2069_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\inmcm4_historical_pr_1970-1999_stats.csv
opening X:\projects\nicaragua\msd\timeseries_results\miro

In [10]:
#produce mean historical values from all 15 GCMs, write output
histmean_all['onset_dy']=hist_onset_dy_15.mean(axis=1)
histmean_all['end_dy']=hist_end_dy_15.mean(axis=1)
histmean_all['duration']=hist_duration_15.mean(axis=1)
histmean_all['intensity']=hist_intensity_15.mean(axis=1)
histmean_all['min']=hist_min_mn_15.mean(axis=1)
histmean_all=pd.concat([cellinfo,histmean_all],axis=1,join_axes=[histmean_all.index])
fname="%s\%s_%s_%s_stats.csv" % (dir,'allgcms_mean','historical','1970-1999')
histmean_all.to_csv(fname, index=False, float_format='%.4f')

In [11]:
#add columns names/locations and write out results for GCMs for historical period
hist_onset_mx_15.columns=[gcms]
hist_onset_mx_15=pd.concat([cellinfo,hist_onset_mx_15],axis=1,join_axes=[hist_onset_mx_15.index])
fname="%s\%s_%s_%s_%s_stats.csv" % (dir,'allgcms','historical','onset_mx','1970-1999')
hist_onset_mx_15.to_csv(fname, index=False, float_format='%.4f')
hist_duration_15.columns=[gcms]
hist_duration_15=pd.concat([cellinfo,hist_duration_15],axis=1,join_axes=[hist_duration_15.index])
fname="%s\%s_%s_%s_%s_stats.csv" % (dir,'allgcms','historical','duration','1970-1999')
hist_duration_15.to_csv(fname, index=False, float_format='%.4f')
hist_intensity_15.columns=[gcms]
hist_intensity_15=pd.concat([cellinfo,hist_intensity_15],axis=1,join_axes=[hist_intensity_15.index])
fname="%s\%s_%s_%s_%s_stats.csv" % (dir,'allgcms','historical','intensity','1970-1999')
hist_intensity_15.to_csv(fname, index=False, float_format='%.4f')
hist_min_mn_15.columns=[gcms]
hist_min_mn_15=pd.concat([cellinfo,hist_min_mn_15],axis=1,join_axes=[hist_min_mn_15.index])
fname="%s\%s_%s_%s_%s_stats.csv" % (dir,'allgcms','historical','min_mn','1970-1999')
hist_min_mn_15.to_csv(fname, index=False, float_format='%.4f')

In [12]:
#calc p-values for projected changes
#write summary output of changes: mean and p-values only. Add cell info
futchanges_all['onset_dy']=futchange_onset.mean(axis=1)
futchanges_all['p-onset_dy']=wilcoxon(futchange_onset)
futchanges_all['end_dy']=futchange_end.mean(axis=1)
futchanges_all['p-end_dy']=wilcoxon(futchange_end)
futchanges_all['duration']=futchange_duration.mean(axis=1)
futchanges_all['p-duration']=wilcoxon(futchange_duration)
futchanges_all['intensity']=futchange_intensity.mean(axis=1)
futchanges_all['p-intensity']=wilcoxon(futchange_intensity)
futchanges_all['min']=futchange_min.mean(axis=1)
futchanges_all['p-min']=wilcoxon(futchange_min)

futchanges_all=pd.concat([cellinfo,futchanges_all],axis=1,join_axes=[futchanges_all.index])
fname="%s\%s_%s_%s_%s_stats.csv" % (dir,'allgcms',scen,'meanchange',futperiod)
futchanges_all.to_csv(fname, index=False, float_format='%.4f')